In [87]:
import pandas as pd
import numpy as np
from datetime import datetime

In [88]:
df_homicidios = pd.read_csv('homicidios_limpio.csv')
df_provincias = pd.read_csv('provincias.csv',sep=',', )

In [89]:
df_homicidios['fecha'] = pd.to_datetime(df_homicidios['fecha'], errors='coerce')

In [90]:
# Lista de columnas a categorizar
categorical_columns = ['tipo_de_calle', 'vehiculo_victima', 'vehiculo_acusado', 'rol', 'sexo','gravedad']

# Se itera sobre las columnas y categorizar solo si están presentes en df_homicidios
for column in categorical_columns:
    if column in df_homicidios.columns:
        df_homicidios[column] = df_homicidios[column].astype('category')


In [91]:
# Se convierte a tipo numérico con downcast a integer
df_homicidios['franja_horaria'] = pd.to_numeric(df_homicidios['franja_horaria'], errors='coerce')


df_homicidios['comuna'] = pd.to_numeric(df_homicidios['comuna'], errors='coerce')


df_homicidios['edad'] = pd.to_numeric(df_homicidios['edad'], errors='coerce')


Tasa de homicidios en siniestros viales y comparacion con año anterior (Objetivo reduccion del 10%)

In [92]:
df_homicidios

,id_siniestro,nro_victimas,fecha,franja_horaria,tipo_de_calle,comuna,longitud,latitud,vehiculo_victima,vehiculo_acusado,rol,sexo,edad
0,2016-0001,1,2016-01-01,4.0,avenida,8,-58.475340,-34.687570,moto,auto,conductor,masculino,19.0
1,2016-0002,1,2016-01-02,1.0,gral paz,9,-58.508775,-34.669777,auto,pasajeros,conductor,masculino,70.0
2,2016-0003,1,2016-01-03,7.0,avenida,1,-58.390403,-34.631894,moto,auto,conductor,masculino,30.0
3,2016-0004,1,2016-01-10,0.0,avenida,8,-58.465039,-34.680930,moto,NaN,conductor,masculino,18.0
4,2016-0005,1,2016-01-21,5.0,avenida,1,-58.387183,-34.622466,moto,pasajeros,conductor,masculino,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,2021-12-13,17.0,avenida,7,-58.433538,-34.645616,moto,auto,pasajero_acompañante,femenino,18.0
713,2021-0094,1,2021-12-20,1.0,autopista,9,-58.467398,-34.651178,moto,auto,pasajero_acompañante,femenino,43.0
714,2021-0095,1,2021-12-30,0.0,avenida,11,-58.472934,-34.619847,moto,cargas,conductor,masculino,27.0
715,2021-0096,1,2021-12-15,10.0,avenida,9,-58.470668,-34.650217,auto,cargas,conductor,masculino,60.0


In [93]:
poblacion_CABA = 3121707

In [94]:
# Se encuentra la última fecha en tu DataFrame
ultima_fecha = df_homicidios['fecha'].max()

# Se calcula la fecha de inicio del semestre actual y anterior
semestre_actual_inicio = ultima_fecha - pd.DateOffset(months=6)
semestre_anterior_inicio = ultima_fecha - pd.DateOffset(months=12)

# Se filtra los datos para el semestre actual y anterior
datos_semestre_actual = df_homicidios[(df_homicidios['fecha'] >= semestre_actual_inicio) & (df_homicidios['fecha'] <= ultima_fecha)]
datos_semestre_anterior = df_homicidios[(df_homicidios['fecha'] >= semestre_anterior_inicio) & (df_homicidios['fecha'] <= semestre_actual_inicio)]

# Se calcula la tasa de homicidios para el semestre actual y anterior
tasa_homicidios_actual = (datos_semestre_actual['nro_victimas'].sum() / poblacion_CABA) * 100000
tasa_homicidios_anterior = (datos_semestre_anterior['nro_victimas'].sum() / poblacion_CABA) * 100000

# Se calcula la reducción porcentual
reduccion_porcentual = ((tasa_homicidios_anterior - tasa_homicidios_actual) / tasa_homicidios_anterior) * 100

if reduccion_porcentual >= 10:
    condicion=(f"La reduccion de tasa de homicidios en siniestros viales respecto al semestre anterior cumple con el objetivo del 10%")
else:
    condicion=(f"La reduccion de tasa de homicidios en siniestros viales respecto al semestre anterior no cumple con el objetivo del 10%")


print(f"Fecha de inicio del semestre actual: {semestre_actual_inicio}")
print(f"Fecha de inicio del semestre anterior: {semestre_anterior_inicio}")
print(f"Tasa de homicidios en siniestros viales del semestre anterior: {tasa_homicidios_anterior:.2f}")
print(f"Tasa de homicidios en siniestros viales del semestre actual: {tasa_homicidios_actual:.2f}")
print(f"Reducción porcentual: {reduccion_porcentual:.2f}%")
print(condicion)

Fecha de inicio del semestre actual: 2021-06-30 00:00:00
Fecha de inicio del semestre anterior: 2020-12-30 00:00:00
Tasa de homicidios en siniestros viales del semestre anterior: 1.76
Tasa de homicidios en siniestros viales del semestre actual: 1.35
Reducción porcentual: 23.64%
La reduccion de tasa de homicidios en siniestros viales respecto al semestre anterior cumple con el objetivo del 10%


Accidentes mortales de motociclistas en el ultimo año, respecto al año anterior (objetivo reduccion del 7%)

In [95]:
# Se filtra los datos solo para motociclistas
accidentes_moto = df_homicidios[df_homicidios['vehiculo_victima'] == 'moto']

# Se encuentra la última fecha en tu DataFrame
ultima_fecha = accidentes_moto['fecha'].max()

# Se calcula la fecha de inicio del año actual y del año anterior
anio_actual_inicio = ultima_fecha - pd.DateOffset(years=1)
anio_anterior_inicio = ultima_fecha - pd.DateOffset(years=2)

# Se filtra los datos para el último año y el año anterior
datos_anio_actual = accidentes_moto[(accidentes_moto['fecha'] >= anio_actual_inicio) & (accidentes_moto['fecha'] <= ultima_fecha)]
datos_anio_anterior = accidentes_moto[(accidentes_moto['fecha'] >= anio_anterior_inicio) & (accidentes_moto['fecha'] <= anio_actual_inicio)]

# Se cuenta la cantidad de accidentes mortales con motociclistas por año
accidentes_actual = len(datos_anio_actual)
accidentes_anterior = len(datos_anio_anterior)

# Se calcula la reducción porcentual
reduccion_porcentual = ((accidentes_anterior - accidentes_actual) / accidentes_anterior) * 100

# Se cuenta la cantidad de accidentes mortales con motociclistas por año
accidentes_actual = len(datos_anio_actual)
accidentes_anterior = len(datos_anio_anterior)

# Se calcula la reducción porcentual
reduccion_porcentual = ((accidentes_anterior - accidentes_actual) / accidentes_anterior) * 100

if reduccion_porcentual >= 7:
    condicion_moto=(f"La reduccion de homicidios en siniestros viales en moto respecto al año anterior cumple con el objetivo del 7%")
else:
    condicion_moto=(f"La reduccion de homicidios en siniestros viales en moto respecto al año anterior no cumple con el objetivo del 7%")


print(f"Fecha de inicio del año actual: {anio_actual_inicio}")
print(f"Fecha de inicio del año anterior: {anio_anterior_inicio}")
print(f"Cantidad de homicidios en moto en el año anterior: {accidentes_anterior:.2f}")
print(f"Cantidad de homicidios en moto en el año actual: {accidentes_actual:.2f}")
print(f"Reducción porcentual: {reduccion_porcentual:.2f}%")
print(condicion_moto)

Fecha de inicio del año actual: 2020-12-30 00:00:00
Fecha de inicio del año anterior: 2019-12-30 00:00:00
Cantidad de homicidios en moto en el año anterior: 28.00
Cantidad de homicidios en moto en el año actual: 46.00
Reducción porcentual: -64.29%
La reduccion de homicidios en siniestros viales en moto respecto al año anterior no cumple con el objetivo del 7%


Tasa de mortalidad en siniestros viales CABA vs provincias Argentinas (año 2019)

Fuente(Pagina 72):

https://www.argentina.gob.ar/sites/default/files/2018/12/ansv_informe_seguridad_vial_con_perspectiva_de_genero_diciembre_2021.pdf

In [96]:
df_provincias['año'] = 2019

In [97]:
# Se crea un diccionario con los datos de la nueva fila
nueva_fila = {'Provincia': 'CABA', 'año': 2019, 'tasa_mortalidad': 2.79}

# Se convierte el diccionario en un DataFrame temporal
df_nueva_fila = pd.DataFrame([nueva_fila])

# Se concatena el DataFrame original con el DataFrame de la nueva fila
df_provincias = pd.concat([df_provincias, df_nueva_fila], ignore_index=True)

In [98]:
df_filtrado = df_provincias.sort_values(ascending=True, by='tasa_mortalidad',ignore_index=True)
print("Top 10 provincias con tasa de mortalidad mas baja")
df_filtrado.head(10)

Top 10 provincias con tasa de mortalidad mas baja


,Provincia,tasa_mortalidad,año
0,CABA,2.79,2019
1,Tierra_del_Fuego,5.30,2019
2,Santa_Cruz,8.40,2019
3,Chubut,8.50,2019
4,Cordoba,9.90,2019
5,Entre_Rios,10.10,2019
6,Rio_Negro,11.50,2019
7,La_Pampa,11.50,2019
8,Salta,11.70,2019
9,Catamarca,12.50,2019


In [99]:
# Se encuentra la última fecha en tu DataFrame
ultima_fecha = df_homicidios['fecha'].max()

# Se calcula la fecha de inicio del semestre actual y anterior
anio_2020_inicio = ultima_fecha - pd.DateOffset(years=1)
anio_2019_inicio = ultima_fecha - pd.DateOffset(years=2)



# Se filtran los datos para el semestre actual y anterior
datos_anio_2019 = df_homicidios[(df_homicidios['fecha'] >= anio_2019_inicio) & (df_homicidios['fecha'] <= anio_2020_inicio)]


# Se calcula la tasa de homicidios para el semestre actual y anterior
tasa_homicidios_2019 = (datos_anio_2019['nro_victimas'].sum() / poblacion_CABA) * 100000


# Se calcula la media de tasa de homicidio del 2019. 
media_tasa_homicidio2019 = df_provincias['tasa_mortalidad'].mean()

# Se calcula KPI performance tasa de homicidios CABA vs resto del pais
tercer_kpi = (tasa_homicidios_2019 / media_tasa_homicidio2019) * 100

# Imprime los resultados

print(f"Performance tasa de homicidios CABA vs resto del pais: {tercer_kpi:.2f}%")
print(f"Tasa de homicidios en siniestros viales del 2019: {tasa_homicidios_2019:.2f}")


Performance tasa de homicidios CABA vs resto del pais: 21.55%
Tasa de homicidios en siniestros viales del 2019: 2.79


In [100]:
df_filtrado = df_provincias.sort_values(ascending=True, by='tasa_mortalidad',ignore_index=True)
print("Top 10 provincias con tasa de mortalidad mas baja")
df_filtrado.head(10)

Top 10 provincias con tasa de mortalidad mas baja


,Provincia,tasa_mortalidad,año
0,CABA,2.79,2019
1,Tierra_del_Fuego,5.30,2019
2,Santa_Cruz,8.40,2019
3,Chubut,8.50,2019
4,Cordoba,9.90,2019
5,Entre_Rios,10.10,2019
6,Rio_Negro,11.50,2019
7,La_Pampa,11.50,2019
8,Salta,11.70,2019
9,Catamarca,12.50,2019


In [101]:
df_provincias.to_csv('df_provincias.csv')